#  ANOVA

In statistics, **Analysis of Variance (ANOVA)** is also used to analyze the differences among group means. The difference between t-test and ANOVA is the former is ued to compare two groups whereas the latter is used to compare three or more groups. [Read more about the difference between t-test and ANOVA](http://b.link/anova24).

From the ANOVA test, you receive two numbers. The first number is called the **F-value** which indicates whether your null-hypothesis can be rejected. The critical F-value that rejects the null-hypothesis varies according to the number of total subjects and the number of subject groups in your experiment. In [this table](http://b.link/eda14) you can find the critical values of the F distribution. **If you are confused by the massive F-distribution table, don't worry. Skip F-value for now and study it at a later time. In this challenge you only need to look at the p-value.**

The p-value is another number yielded by ANOVA which already takes the number of total subjects and the number of experiment groups into consideration. **Typically if your p-value is less than 0.05, you can declare the null-hypothesis is rejected.**

In this challenge, we want to understand whether there are significant differences among various types of pokemons' `Total` value, i.e. Grass vs Poison vs Fire vs Dragon... There are many types of pokemons which makes it a perfect use case for ANOVA. 

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import scipy.stats as st

In [2]:
# Load the data:
df = pd.read_csv('pokemon.txt', sep = ',')
df.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


In [3]:
df.shape

(800, 13)

**To achieve our goal, we use three steps:**

1. **Extract the unique values of the pokemon types.**

1. **Select dataframes for each unique pokemon type.**

1. **Conduct ANOVA analysis across the pokemon types.**

#### First let's obtain the unique values of the pokemon types. These values should be extracted from Type 1 and Type 2 aggregated. Assign the unique values to a variable called `unique_types`.

*Hint: the correct number of unique types is 19 including `NaN`. You can disregard `NaN` in next step.*

In [4]:
# Creating an aggregated type column:
# I do the step in between with the set to eliminate the duplicates
unique_types = list(set(list(df['Type 1'].unique()) + list(df['Type 2'].unique())))
unique_types

[nan,
 'Fighting',
 'Dark',
 'Poison',
 'Bug',
 'Psychic',
 'Grass',
 'Normal',
 'Electric',
 'Water',
 'Dragon',
 'Fairy',
 'Rock',
 'Ice',
 'Fire',
 'Ghost',
 'Ground',
 'Steel',
 'Flying']

In [5]:
# Check if it's 19 (hint)
len(unique_types)

19

#### Second we will create a list named `pokemon_totals` to contain the `Total` values of each unique type of pokemons.

Why we use a list instead of a dictionary to store the pokemon `Total`? It's because ANOVA only tells us whether there is a significant difference of the group means but does not tell which group(s) are significantly different. Therefore, we don't need know which `Total` belongs to which pokemon type.

*Hints:*

* Loop through `unique_types` and append the selected type's `Total` to `pokemon_groups`. Be sure to loop through BOTH `Type 1` and `Type 2` to cover all occurrances of each unique type.
* Skip the `NaN` value in `unique_types`. `NaN` is a `float` variable which you can find out by using `type()`. The valid pokemon type values are all of the `str` type.
* At the end, the length of your `pokemon_totals` should be 18.

In [6]:
# Remove the "nan" value from the list
unique_types = [x for x in unique_types if str(x) != 'nan']
len(unique_types)

18

In [7]:
# Create a list to store the Total values for each unique Pokemon type
pokemon_totals = []

# Loop through each unique type
for typ in unique_types:
    # Append the Total values for the current type from both 'Type 1' and 'Type 2'
    pokemon_totals.append(list(df[(df['Type 1'] == typ)  | (df['Type 2'] == typ)]['Total']))

In [8]:
# Check the length of pokemon_totals to verify it's 18 as expected
len(pokemon_totals)

18

In [9]:
# Convert the 'pokemon_totals' list to a DataFrame to check how it looks like
# Each sublist in 'pokemon_totals' will be a column in the DataFrame

# Create a dictionary to ensure that each Pokémon type is the key and the totals are the values
pokemon_totals_dict = {type_: totals for type_, totals in zip(unique_types, pokemon_totals) if pd.notna(type_)}

# Convert this dictionary into a DataFrame, where each dictionary key will be a column
pokemon_totals_df = pd.DataFrame.from_dict(pokemon_totals_dict, orient='index').transpose()

# Display the resulting DataFrame
pokemon_totals_df.head()

,Fighting,Dark,Poison,Bug,Psychic,Grass,Normal,Electric,Water,Dragon,Fairy,Rock,Ice,Fire,Ghost,Ground,Steel,Flying
0,305.0,640.0,318.0,195.0,310.0,318.0,251.0,320.0,314.0,634.0,323.0,300.0,475.0,309.0,310.0,300.0,325.0,534.0
1,455.0,525.0,405.0,205.0,400.0,405.0,349.0,485.0,405.0,300.0,483.0,390.0,525.0,405.0,405.0,450.0,465.0,634.0
2,510.0,405.0,525.0,395.0,500.0,525.0,479.0,325.0,530.0,420.0,270.0,495.0,455.0,534.0,500.0,505.0,465.0,395.0
3,305.0,430.0,625.0,195.0,590.0,625.0,579.0,465.0,630.0,600.0,435.0,385.0,535.0,634.0,600.0,505.0,510.0,251.0
4,405.0,330.0,195.0,205.0,315.0,320.0,253.0,330.0,320.0,610.0,460.0,345.0,580.0,634.0,435.0,265.0,610.0,349.0


In [10]:
pokemon_totals_df.shape

(126, 18)

#### Now we run ANOVA test on `pokemon_totals`.

*Hints:*

* To conduct ANOVA, you can use `scipy.stats.f_oneway()`. Here's the [reference](http://b.link/scipy44).

* What if `f_oneway` throws an error because it does not accept `pokemon_totals` as a list? The trick is to add a `*` in front of `pokemon_totals`, e.g. `stats.f_oneway(*pokemon_groups)`. This trick breaks the list and supplies each list item as a parameter for `f_oneway`.

In [11]:
# H0: pokemon totals are equal around all pokemon types
# H1: totals for the different pokemon types are different

st.f_oneway(*pokemon_totals)

F_onewayResult(statistic=6.617538296005532, pvalue=2.6457458815984803e-15)

#### Interpret the ANOVA test result. Is the difference significant?

Since the p-value is much less than 0.05, we would reject the null hypothesis. 

This means that there is strong evidence to suggest that the means of the different Pokémon types are not all equal and that there are statistically significant differences between them.